In [1]:
%pip install scipy
%pip install statsmodels
%pip install ace-tools
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import statsmodels.api as sm


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Load the dataset
file_path = "AP PW1 Data Cleaned.csv" #REPLACE WITH YOUR OWN PATH
df = pd.read_csv(file_path)

# Rename the first column to 'Date'
df.rename(columns={df.columns[0]: 'Date'}, inplace=True)

# Load market return dataset
market_file_path = "Market Return Cleaned.csv" #REPLACE WITH YOUR OWN PATH
market_df = pd.read_csv(market_file_path)

market_df.rename(columns={market_df.columns[0]: 'Date'}, inplace=True)

# Merge datasets on Date
merged_df = pd.merge(df, market_df, on='Date')

merged_df

,Date,ME1 BM1,ME1 BM2,ME1 BM3,ME1 BM4,ME1 BM5,ME2 BM1,ME2 BM2,ME2 BM3,ME2 BM4,...,ME4 BM3,ME4 BM4,ME4 BM5,ME5 BM1,ME5 BM2,ME5 BM3,ME5 BM4,ME5 BM5,Market Return,Rf
0,196307,1.1307,-0.3091,0.7079,0.1062,-1.3211,-1.8071,0.1899,-1.0105,-1.9644,...,-1.9184,-1.5741,-1.8567,0.1547,0.4833,1.2286,-0.5862,-1.1026,-0.66,0.27
1,196308,4.2370,1.3834,1.4977,2.3755,4.7567,5.5665,4.5191,4.4424,4.4188,...,6.2332,7.6782,5.3469,5.7691,4.2550,4.5936,8.2831,6.3824,4.82,0.25
2,196309,-2.8878,0.6263,-1.0204,-1.6000,-0.4320,-4.0502,-1.5034,-0.8798,-1.1812,...,-1.7800,-3.9639,-1.9943,-1.3595,-0.8054,-0.8135,-0.2145,-3.4963,-1.84,0.27
3,196310,1.2885,-0.7071,1.3132,0.0855,2.3988,1.1916,4.2342,2.3524,2.2012,...,2.6229,4.8492,0.6113,5.3339,1.7427,-0.2469,2.3915,0.4857,2.24,0.29
4,196311,-3.3751,-3.7534,-1.8055,-1.0487,-1.0538,-4.2561,-1.7534,-0.7809,-0.1002,...,-0.7933,1.3610,3.5388,-1.2556,1.0072,-1.7425,-2.0838,1.3455,-1.12,0.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,202402,9.7415,6.8190,11.7272,2.6284,4.2753,9.6645,6.6236,7.1860,4.3233,...,3.7097,4.6091,2.5429,6.0699,1.5096,8.0769,4.9935,2.9089,4.65,0.42
728,202403,-3.2823,1.5975,0.7470,1.6994,4.7100,0.1016,2.4109,2.5531,2.8258,...,5.1387,8.1868,7.6653,1.6713,6.2897,3.6674,6.1407,8.9863,2.40,0.43
729,202404,-12.5428,-7.1861,-7.5104,-7.4424,-7.5468,-6.8279,-8.2722,-6.3410,-5.6108,...,-6.0023,-6.8561,-5.7337,-3.9996,1.1908,-4.4722,-4.9938,-5.4745,-5.14,0.47
730,202405,7.2793,3.2521,4.9308,3.5565,4.7890,4.5944,3.2302,5.5056,4.8427,...,2.3887,5.2926,3.6067,5.9812,4.1988,3.0522,4.0484,4.2659,3.90,0.44


### Question 1a

In [ ]:
# Drop the 'Date' column for calculations
data = merged_df.drop(columns=['Date'])

# Extract size and B/M quintiles from column names
column_names = [col.split() for col in data.columns if col not in ['Market Return', 'Rf']]
size_quintile_labels = [name[0] for name in column_names]
bm_quintile_labels = [name[1] if len(name) > 1 else '' for name in column_names]

# Compute arithmetic average monthly returns
average_returns = data.drop(columns=['Market Return', 'Rf']).mean()
average_returns_df = pd.DataFrame(average_returns, columns=['Avg Monthly Return'])
average_returns_df['Size Quintile'] = size_quintile_labels
average_returns_df['B/M Quintile'] = bm_quintile_labels

# Pivot the table to have size in rows and B/M quintiles in columns
average_returns_pivot = average_returns_df.pivot(index='Size Quintile', columns='B/M Quintile', values='Avg Monthly Return')

# Use 'Market Return' for beta calculations
market_returns = data['Market Return']
betas = {}

for col in data.columns:
    if col not in ['Market Return', 'Rf']:  # Avoid regressing market on itself
        X = sm.add_constant(market_returns)
        y = data[col]
        model = sm.OLS(y, X).fit()
        betas[col] = model.params[1]  # Extract beta coefficient

# Convert results to DataFrame
betas_df = pd.DataFrame.from_dict(betas, orient='index', columns=['Market Beta'])
betas_df['Size Quintile'] = size_quintile_labels
betas_df['B/M Quintile'] = bm_quintile_labels

# Pivot the table for betas
betas_pivot = betas_df.pivot(index='Size Quintile', columns='B/M Quintile', values='Market Beta')

# Display tables
print("Arithmetic Average Monthly Returns:")
print(average_returns_pivot)
print("\nMarket Betas:")
print(betas_pivot)


Arithmetic Average Monthly Returns:
B/M Quintile        BM1       BM2       BM3       BM4       BM5
Size Quintile                                                  
ME1            0.626547  1.122644  1.126640  1.316065  1.452358
ME2            0.876701  1.137901  1.216862  1.248196  1.356403
ME3            0.899527  1.159793  1.108411  1.233549  1.344528
ME4            1.009955  1.015300  1.073193  1.193704  1.225115
ME5            0.967251  0.923706  0.952686  0.888321  1.045587

Market Betas:
B/M Quintile        BM1       BM2       BM3       BM4       BM5
Size Quintile                                                  
ME1            1.395704  1.205931  1.094002  1.021771  1.054580
ME2            1.362312  1.165658  1.060227  1.003788  1.131298
ME3            1.286599  1.106563  0.997735  0.987330  1.070450
ME4            1.183100  1.065877  0.994796  0.981917  1.063999
ME5            0.971703  0.904358  0.845979  0.871516  0.965705


/tmp/ipykernel_15122/3598909247.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  betas[col] = model.params[1]  # Extract beta coefficient
/tmp/ipykernel_15122/3598909247.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  betas[col] = model.params[1]  # Extract beta coefficient
/tmp/ipykernel_15122/3598909247.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  betas[col] = model.params[1]  # Extract beta coefficient
/tmp/ipyke

In [7]:
betas_pivot

B/M Quintile,BM1,BM2,BM3,BM4,BM5
Size Quintile,,,,,
ME1,1.395704,1.205931,1.094002,1.021771,1.054580
ME2,1.362312,1.165658,1.060227,1.003788,1.131298
ME3,1.286599,1.106563,0.997735,0.987330,1.070450
ME4,1.183100,1.065877,0.994796,0.981917,1.063999
ME5,0.971703,0.904358,0.845979,0.871516,0.965705


In [8]:
average_returns_pivot

B/M Quintile,BM1,BM2,BM3,BM4,BM5
Size Quintile,,,,,
ME1,0.626547,1.122644,1.126640,1.316065,1.452358
ME2,0.876701,1.137901,1.216862,1.248196,1.356403
ME3,0.899527,1.159793,1.108411,1.233549,1.344528
ME4,1.009955,1.015300,1.073193,1.193704,1.225115
ME5,0.967251,0.923706,0.952686,0.888321,1.045587


## Question 1b

In [9]:
# Compute time-series standard deviations of portfolio returns
std_devs = data.drop(columns=['Market Return', 'Rf']).std()
std_devs_df = pd.DataFrame(std_devs, columns=['Time-Series Std Dev'])
std_devs_df['Size Quintile'] = size_quintile_labels
std_devs_df['B/M Quintile'] = bm_quintile_labels
std_devs_pivot = std_devs_df.pivot(index='Size Quintile', columns='B/M Quintile', values='Time-Series Std Dev')

# Compute Sharpe Ratios using the new risk-free rate column
sharpe_ratios = (average_returns - data['Rf'].mean()) / std_devs
sharpe_ratios_df = pd.DataFrame(sharpe_ratios, columns=['Sharpe Ratio'])
sharpe_ratios_df['Size Quintile'] = size_quintile_labels
sharpe_ratios_df['B/M Quintile'] = bm_quintile_labels
sharpe_ratios_pivot = sharpe_ratios_df.pivot(index='Size Quintile', columns='B/M Quintile', values='Sharpe Ratio')

print("\nTime-Series Standard Deviations:")
print(std_devs_pivot)
print("\nSharpe Ratios:")
print(sharpe_ratios_pivot)



Time-Series Standard Deviations:
B/M Quintile        BM1       BM2       BM3       BM4       BM5
Size Quintile                                                  
ME1            8.013224  6.960714  6.087275  5.850233  6.300806
ME2            7.186735  6.093141  5.563411  5.385569  6.271112
ME3            6.564381  5.539268  5.082641  5.176016  5.945698
ME4            5.843349  5.179828  5.008133  5.097439  5.840597
ME5            4.699530  4.406206  4.341430  4.772681  5.684939

Sharpe Ratios:
B/M Quintile        BM1       BM2       BM3       BM4       BM5
Size Quintile                                                  
ME1            0.032844  0.109081  0.125389  0.162849  0.172835
ME2            0.071429  0.127117  0.153413  0.164297  0.158352
ME3            0.081678  0.143779  0.146587  0.168119  0.165021
ME4            0.110655  0.125861  0.141736  0.162894  0.147546
ME5            0.128500  0.127172  0.135744  0.109993  0.120006


## Question 1c

In [21]:
print(t_stats_5_minus_1_df.shape)
print(t_stats_pivot.shape)

(10, 1)
(5, 5)


In [20]:
### Compute T-Statistics for Portfolio Returns ###
t_stats = (average_returns / (std_devs / np.sqrt(len(data))))
t_stats_df = pd.DataFrame(t_stats, columns=['T-Statistic'])
t_stats_df['Size Quintile'] = size_quintile_labels
t_stats_df['B/M Quintile'] = bm_quintile_labels
t_stats_pivot = t_stats_df.pivot(index='Size Quintile', columns='B/M Quintile', values='T-Statistic')

In [22]:
### Compute T-Statistics for Quintile 5 Minus 1 ###
t_stats_5_minus_1 = {}

# Compute for each B/M quintile (ME5 - ME1)
for bm in set(bm_quintile_labels):
    me1_col = [col for col in data.columns if f'ME1 {bm}' in col][0]
    me5_col = [col for col in data.columns if f'ME5 {bm}' in col][0]
    diff = data[me5_col] - data[me1_col]
    t_stats_5_minus_1[f'ME5 - ME1 {bm}'] = diff.mean() / (diff.std() / np.sqrt(len(data)))

# Compute for each Size quintile (BM5 - BM1)
for me in set(size_quintile_labels):
    bm1_col = [col for col in data.columns if f'{me} BM1' in col][0]
    bm5_col = [col for col in data.columns if f'{me} BM5' in col][0]
    diff = data[bm5_col] - data[bm1_col]
    t_stats_5_minus_1[f'{me} BM5 - BM1'] = diff.mean() / (diff.std() / np.sqrt(len(data)))

# Convert results to DataFrame
t_stats_5_minus_1_df = pd.DataFrame(t_stats_5_minus_1, index=['Quintile 5 - 1']).T

# Append to the main t-stats table
t_stats_pivot = pd.concat([t_stats_pivot, t_stats_5_minus_1_df.T])

print("\nT-Statistics:")
print(t_stats_pivot)





T-Statistics:
                     BM1       BM2       BM3       BM4       BM5  \
ME1             2.115447  4.363589  5.007464  6.086391  6.236387   
ME2             3.300468  5.052644  5.917738  6.270565  5.851937   
ME3             3.707456  5.664789  5.900201  6.447870  6.118184   
ME4             4.676231  5.303157  5.797726  6.335780  5.675122   
ME5             5.568529  5.671848  5.937072  5.035738  4.976110   
Quintile 5 - 1       NaN       NaN       NaN       NaN       NaN   

                ME5 - ME1 BM3  ME5 - ME1 BM5  ME5 - ME1 BM2  ME5 - ME1 BM4  \
ME1                       NaN            NaN            NaN            NaN   
ME2                       NaN            NaN            NaN            NaN   
ME3                       NaN            NaN            NaN            NaN   
ME4                       NaN            NaN            NaN            NaN   
ME5                       NaN            NaN            NaN            NaN   
Quintile 5 - 1      -1.048958      -2.23